<a href="https://colab.research.google.com/github/alexanderwilly/ML_content_moderation/blob/main/Jigsaw_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!rm -rf /content/nltk_data

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Data Collection & Preparation

In [ ]:
import os
print(os.getcwd())

/content


In [ ]:
!pip install xgboost

In [ ]:
!pip install nltk

In [ ]:
!pip install --upgrade datasets -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-nvrtc-cu12 12.5.82 which is incompatible.
torch 2.6

In [ ]:
import nltk
import os

# Define a custom NLTK data path (e.g., in your home directory or current project directory)
# For example, to create it in your current working directory:
custom_nltk_data_path = os.path.join(os.getcwd(), "nltk_data") # Creates a folder "nltk_data" in your current notebook directory

# If you prefer to download to a user-specific folder (e.g., your home directory):
# custom_nltk_data_path = os.path.join(os.path.expanduser("~"), "nltk_data")

if not os.path.exists(custom_nltk_data_path):
    os.makedirs(custom_nltk_data_path)
    print(f"Created NLTK data directory: {custom_nltk_data_path}")

nltk.data.path.append(custom_nltk_data_path)
print(f"NLTK data search paths: {nltk.data.path}")


try:
    nltk.download('stopwords', download_dir=custom_nltk_data_path)
    nltk.download('punkt', download_dir=custom_nltk_data_path)
    nltk.download('wordnet', download_dir=custom_nltk_data_path)
    print("NLTK data downloaded to specified path.")
except Exception as e:
    print(f"Error during NLTK download: {e}")

# After running this, restart your kernel (Kernel -> Restart Kernel)
# Then, try running the main model training code again.

Created NLTK data directory: /content/nltk_data
NLTK data search paths: ['/root/nltk_data', '/usr/nltk_data', '/usr/share/nltk_data', '/usr/lib/nltk_data', '/usr/share/nltk_data', '/usr/local/share/nltk_data', '/usr/lib/nltk_data', '/usr/local/lib/nltk_data', '/content/nltk_data']


[nltk_data] Downloading package stopwords to /content/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /content/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /content/nltk_data...


NLTK data downloaded to specified path.


In [ ]:
df = pd.read_csv('/content/Dataset/Jigsaw/train.csv')
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [ ]:
# Initialize vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english')

### Toxic Label

In [ ]:
toxic_dfs = df[df['toxic'] == 1]
toxic_dfs.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
12,0005c987bdfc9d4b,Hey... what is it..\n@ | talk .\nWhat is it......,1,0,0,0,0,0
16,0007e25b2121310b,"Bye! \n\nDon't look, come or think of comming ...",1,0,0,0,0,0
42,001810bf8c45bf5f,You are gay or antisemmitian? \n\nArchangel WH...,1,0,1,0,1,1
43,00190820581d90ce,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",1,0,1,0,1,0


In [ ]:
toxic_dfs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15294 entries, 6 to 159554
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             15294 non-null  object
 1   comment_text   15294 non-null  object
 2   toxic          15294 non-null  int64 
 3   severe_toxic   15294 non-null  int64 
 4   obscene        15294 non-null  int64 
 5   threat         15294 non-null  int64 
 6   insult         15294 non-null  int64 
 7   identity_hate  15294 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 1.1+ MB


In [ ]:
for i in range(5):
    print(toxic_dfs.iloc[i]['comment_text'])
    print('=====')

COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK
=====
Hey... what is it..
@ | talk .
What is it... an exclusive group of some WP TALIBANS...who are good at destroying, self-appointed purist who GANG UP any one who asks them questions abt their ANTI-SOCIAL and DESTRUCTIVE (non)-contribution at WP?

Ask Sityush to clean up his behavior than issue me nonsensical warnings...
=====
Bye! 

Don't look, come or think of comming back! Tosser.
=====
You are gay or antisemmitian? 

Archangel WHite Tiger

Meow! Greetingshhh!

Uh, there are two ways, why you do erased my comment about WW2, that holocaust was brutally slaying of Jews and not gays/Gypsys/Slavs/anyone...

1 - If you are anti-semitian, than shave your head bald and go to the skinhead meetings!

2 - If you doubt words of the Bible, that homosexuality is a deadly sin, make a pentagram tatoo on your forehead go to the satanistic masses with your gay pals!

3 - First and last warning, you fucking gay - I won't appreciate if any more nazi shwa

In [ ]:
# Fit and transform the comments column
tfidf_matrix = tfidf_vectorizer.fit_transform(toxic_dfs['comment_text'])

# Convert to DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Sum TF-IDF scores for each word
word_importance = tfidf_df.sum(axis=0).sort_values(ascending=False)

# Display top 20 words
top_n = 20
print(word_importance.head(top_n))

fuck         481.515289
fucking      287.459738
wikipedia    240.715460
like         238.607253
don          235.349667
just         214.595215
shit         211.992420
page         206.782314
stop         176.959349
bitch        171.230804
suck         169.240549
ass          162.929772
know         162.683611
stupid       160.581136
people       155.655242
talk         146.134755
asshole      137.567427
gay          135.493252
article      132.178514
think        131.058245
dtype: float64


### Severe Toxic

In [ ]:
severe_toxic_dfs = df[df['severe_toxic'] == 1]
severe_toxic_dfs.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
55,0020e7119b96eeeb,Stupid peace of shit stop deleting my stuff as...,1,1,1,0,1,0
181,006e87872c8b370c,you are a stupid fuck \n\nand your mother's cu...,1,1,1,0,1,0
442,01208d2b76624130,Hi \n\nIm a fucking bitch.\n\n50.180.208.181,1,1,1,0,1,0
579,018663f910e0bfe6,What a motherfucking piece of crap those fuckh...,1,1,1,0,1,0


In [ ]:
for i in range(5):
    print(severe_toxic_dfs.iloc[i]['comment_text'])
    print('=====')

COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK
=====
Stupid peace of shit stop deleting my stuff asshole go die and fall in a hole go to hell!
=====
you are a stupid fuck 

and your mother's cunt stinks
=====
Hi 

Im a fucking bitch.

50.180.208.181
=====
What a motherfucking piece of crap those fuckheads for blocking us!
=====


In [ ]:
# Fit and transform the comments column
tfidf_matrix = tfidf_vectorizer.fit_transform(severe_toxic_dfs['comment_text'])

# Convert to DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Sum TF-IDF scores for each word
word_importance = tfidf_df.sum(axis=0).sort_values(ascending=False)

# Display top 20 words
top_n = 20
print(word_importance.head(top_n))

fuck         134.628677
fucking       69.114172
bitch         52.301755
suck          46.045705
shit          45.331617
ass           37.955350
asshole       35.911765
dick          34.230885
cunt          32.400244
faggot        31.435279
wikipedia     23.315457
cock          22.704415
like          22.058268
stupid        21.824848
gay           21.052647
die           20.912601
page          20.491765
piece         20.309491
don           19.196460
nigger        18.926190
dtype: float64


### Obscene

In [ ]:
obscene_dfs = df[df['obscene'] == 1]
obscene_dfs.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
42,001810bf8c45bf5f,You are gay or antisemmitian? \n\nArchangel WH...,1,0,1,0,1,1
43,00190820581d90ce,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",1,0,1,0,1,0
51,001dc38a83d420cf,GET FUCKED UP. GET FUCKEEED UP. GOT A DRINK T...,1,0,1,0,0,0
55,0020e7119b96eeeb,Stupid peace of shit stop deleting my stuff as...,1,1,1,0,1,0


In [ ]:
for i in range(5):
    print(obscene_dfs.iloc[i]['comment_text'])
    print('=====')

COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK
=====
You are gay or antisemmitian? 

Archangel WHite Tiger

Meow! Greetingshhh!

Uh, there are two ways, why you do erased my comment about WW2, that holocaust was brutally slaying of Jews and not gays/Gypsys/Slavs/anyone...

1 - If you are anti-semitian, than shave your head bald and go to the skinhead meetings!

2 - If you doubt words of the Bible, that homosexuality is a deadly sin, make a pentagram tatoo on your forehead go to the satanistic masses with your gay pals!

3 - First and last warning, you fucking gay - I won't appreciate if any more nazi shwain would write in my page! I don't wish to talk to you anymore!

Beware of the Dark Side!
=====
FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!
=====
GET FUCKED UP. GET FUCKEEED UP.  GOT A DRINK THAT YOU CANT PUT DOWN???/ GET FUCK UP GET FUCKED UP.  I'M FUCKED UP RIGHT NOW!
=====
Stupid peace of shit stop deleting my stuff asshole go die and fall in a hole go to hell!
=====


In [ ]:
# Fit and transform the comments column
tfidf_matrix = tfidf_vectorizer.fit_transform(obscene_dfs['comment_text'])

# Convert to DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Sum TF-IDF scores for each word
word_importance = tfidf_df.sum(axis=0).sort_values(ascending=False)

# Display top 20 words
top_n = 20
print(word_importance.head(top_n))

fuck         402.047020
fucking      240.341979
shit         167.225687
bitch        153.047721
suck         146.211233
ass          136.001211
don          131.030816
like         129.175002
wikipedia    128.973991
asshole      121.965857
just         114.545333
page         113.142017
dick         107.382686
cunt         107.154351
faggot        98.687073
stupid        97.984209
stop          91.846019
know          90.086440
talk          77.036394
life          76.819870
dtype: float64


### Threat

In [ ]:
threat_dfs = df[df['threat'] == 1]
threat_dfs.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
79,003217c3eb469ba9,Hi! I am back again!\nLast warning!\nStop undo...,1,0,0,1,0,0
176,006b94add72ed61c,I think that your a Fagget get a oife and burn...,1,0,1,1,1,1
600,0199d6af27b715f3,I'm also a sock puppet of this account...SUPRI...,1,0,0,1,0,0
802,02230885017a50c5,"Fuck you, Smith. Please have me notified when ...",1,0,1,1,1,0
1017,02c6e41e4b317ac3,WOULDN'T BE THE FIRST TIME BITCH. FUCK YOU I'L...,1,1,1,1,1,1


In [ ]:
for i in range(5):
    print(threat_dfs.iloc[i]['comment_text'])
    print('=====')

Hi! I am back again!
Last warning!
Stop undoing my edits or die!
=====
I think that your a Fagget get a oife and burn in Hell I hate you 'm sorry we cant have any more sex i'm running out of conndoms
=====
I'm also a sock puppet of this account...SUPRISE!!
-sincerely,
            The man that will track you down from the Internet and kill you
=====
Fuck you, Smith. Please have me notified when you die. I want to dance on your grave.
=====
WOULDN'T BE THE FIRST TIME BITCH. FUCK YOU I'LL FIND OUT WHERE YOU LIVE, SODOMIZE YOUR WIFE AND THEN BURN YOUR HOUSE DOWN. FUCK YOU YOU FUCKING QUEER.
=====


In [ ]:
# Fit and transform the comments column
tfidf_matrix = tfidf_vectorizer.fit_transform(threat_dfs['comment_text'])

# Convert to DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Sum TF-IDF scores for each word
word_importance = tfidf_df.sum(axis=0).sort_values(ascending=False)

# Display top 20 words
top_n = 20
print(word_importance.head(top_n))

die          22.329723
kill         21.735886
fucking      16.112064
going        15.165183
fuck         14.999617
ass          10.983075
hope         10.017632
ll            9.916491
shit          9.276331
don           8.885255
bitch         7.814591
like          7.261732
rape          7.030021
hell          6.938787
know          6.684880
death         6.636852
just          6.294228
wikipedia     6.196249
im            6.193040
stop          6.065682
dtype: float64


### Insult

In [ ]:
insult_dfs = df[df['insult'] == 1]
insult_dfs.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
42,001810bf8c45bf5f,You are gay or antisemmitian? \n\nArchangel WH...,1,0,1,0,1,1
43,00190820581d90ce,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",1,0,1,0,1,0
55,0020e7119b96eeeb,Stupid peace of shit stop deleting my stuff as...,1,1,1,0,1,0
56,0020fd96ed3b8c8b,=Tony Sidaway is obviously a fistfuckee. He lo...,1,0,1,0,1,0


In [ ]:
for i in range(5):
    print(insult_dfs.iloc[i]['comment_text'])
    print('=====')

COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK
=====
You are gay or antisemmitian? 

Archangel WHite Tiger

Meow! Greetingshhh!

Uh, there are two ways, why you do erased my comment about WW2, that holocaust was brutally slaying of Jews and not gays/Gypsys/Slavs/anyone...

1 - If you are anti-semitian, than shave your head bald and go to the skinhead meetings!

2 - If you doubt words of the Bible, that homosexuality is a deadly sin, make a pentagram tatoo on your forehead go to the satanistic masses with your gay pals!

3 - First and last warning, you fucking gay - I won't appreciate if any more nazi shwain would write in my page! I don't wish to talk to you anymore!

Beware of the Dark Side!
=====
FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!
=====
Stupid peace of shit stop deleting my stuff asshole go die and fall in a hole go to hell!
=====
=Tony Sidaway is obviously a fistfuckee. He loves an arm up his ass.
=====


In [ ]:
# Fit and transform the comments column
tfidf_matrix = tfidf_vectorizer.fit_transform(insult_dfs['comment_text'])

# Convert to DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Sum TF-IDF scores for each word
word_importance = tfidf_df.sum(axis=0).sort_values(ascending=False)

# Display top 20 words
top_n = 20
print(word_importance.head(top_n))

fuck         333.992597
fucking      205.833313
bitch        143.449912
shit         136.426614
like         126.464894
suck         124.834796
wikipedia    122.315550
don          116.662045
ass          114.286093
asshole      111.728485
stupid       108.559108
just         104.683435
page          98.828506
faggot        98.375804
cunt          95.634800
idiot         92.026525
stop          90.705592
dick          86.532372
know          82.789997
life          79.694418
dtype: float64


### Identity Hate

In [ ]:
identity_hate_dfs = df[df['identity_hate'] == 1]
identity_hate_dfs.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
42,001810bf8c45bf5f,You are gay or antisemmitian? \n\nArchangel WH...,1,0,1,0,1,1
105,00472b8e2d38d1ea,A pair of jew-hating weiner nazi schmucks.,1,0,1,0,1,1
176,006b94add72ed61c,I think that your a Fagget get a oife and burn...,1,0,1,1,1,1
218,008e0818dde894fb,"Kill all niggers. \n\nI have hard, that others...",1,0,1,0,1,1
238,0097dd5c29bf7a15,u r a tw@ fuck off u gay boy.U r smelly.Fuck u...,1,0,1,0,1,1


In [ ]:
for i in range(5):
    print(identity_hate_dfs.iloc[i]['comment_text'])
    print('=====')

You are gay or antisemmitian? 

Archangel WHite Tiger

Meow! Greetingshhh!

Uh, there are two ways, why you do erased my comment about WW2, that holocaust was brutally slaying of Jews and not gays/Gypsys/Slavs/anyone...

1 - If you are anti-semitian, than shave your head bald and go to the skinhead meetings!

2 - If you doubt words of the Bible, that homosexuality is a deadly sin, make a pentagram tatoo on your forehead go to the satanistic masses with your gay pals!

3 - First and last warning, you fucking gay - I won't appreciate if any more nazi shwain would write in my page! I don't wish to talk to you anymore!

Beware of the Dark Side!
=====
A pair of jew-hating weiner nazi schmucks.
=====
I think that your a Fagget get a oife and burn in Hell I hate you 'm sorry we cant have any more sex i'm running out of conndoms
=====
Kill all niggers. 

I have hard, that others have said this.. should this be included? That racists sometimes say these.
=====
u r a tw@ fuck off u gay boy.U r s

In [ ]:
# Fit and transform the comments column
tfidf_matrix = tfidf_vectorizer.fit_transform(identity_hate_dfs['comment_text'])

# Convert to DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Sum TF-IDF scores for each word
word_importance = tfidf_df.sum(axis=0).sort_values(ascending=False)

# Display top 20 words
top_n = 20
print(word_importance.head(top_n))

gay          57.160447
fuck         53.162776
faggot       39.784168
fucking      39.346373
nigger       38.891363
shit         27.896039
like         25.683218
bitch        24.335075
ass          22.537983
jew          16.923066
wikipedia    16.885345
just         16.088987
stupid       15.965678
people       15.920474
don          15.342152
cunt         15.239194
fag          14.827887
suck         14.411281
page         14.271350
stop         13.864846
dtype: float64


### All

In [ ]:
all_dfs = df[(df['identity_hate'] == 1) & (df['insult'] == 1) & (df['threat'] == 1) & (df['obscene'] == 1) & (df['severe_toxic'] == 1) & (df['toxic'] == 1) ]
all_dfs.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
1017,02c6e41e4b317ac3,WOULDN'T BE THE FIRST TIME BITCH. FUCK YOU I'L...,1,1,1,1,1,1
1312,039296aa294ee10b,"SHUT UP, YOU FAT POOP, OR I WILL KICK YOUR ASS!!!",1,1,1,1,1,1
7299,1368c10281978876,"You're a stupid cunt \n\nFuck you dumb arse, y...",1,1,1,1,1,1
13648,2405c66d017f139e,Bitch \n\nYou are a little bitch. I fuckin spe...,1,1,1,1,1,1
13964,24d2b50726b67167,I am going to murder ZimZalaBim ST47 for being...,1,1,1,1,1,1


In [ ]:
for i in range(5):
    print(all_dfs.iloc[i]['comment_text'])
    print('=====')

WOULDN'T BE THE FIRST TIME BITCH. FUCK YOU I'LL FIND OUT WHERE YOU LIVE, SODOMIZE YOUR WIFE AND THEN BURN YOUR HOUSE DOWN. FUCK YOU YOU FUCKING QUEER.
=====
SHUT UP, YOU FAT POOP, OR I WILL KICK YOUR ASS!!!
=====
You're a stupid cunt 

Fuck you dumb arse, your mum has a hairy cunt and I hope and pray that you die, no, fuck that, I wish you would die, if I had three wishes, one of them would be to have you dead, and the others would be so that everyone you knew hated your cunt like face.
=====
Bitch 

You are a little bitch. I fuckin spent 6 hours on big sam (rapper) an you call it fuckin vandolism, hard ass aint you, fuck you i'm makin a true ass page an you wack off an delete my page. What the fuck is up, your like a snitch but to yourself. I'll fuckin edit your bitch ass page an delete that shit. I'll fuck up every single thing on your gay ass website if you keep deletin my ass. Wanna talk to me, IM me at GuttyBoy3D or email me at GuttyBoy3D@aol.com
=====
I am going to murder ZimZala

In [ ]:
# Fit and transform the comments column
tfidf_matrix = tfidf_vectorizer.fit_transform(all_dfs['comment_text'])

# Convert to DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Sum TF-IDF scores for each word
word_importance = tfidf_df.sum(axis=0).sort_values(ascending=False)

# Display top 20 words
top_n = 20
print(word_importance.head(top_n))

fucking    2.522879
fuck       2.412048
die        2.357157
ass        2.021699
bitch      1.763611
rape       1.674162
shit       1.321384
jew        1.280964
fuckin     1.203862
ll         1.159874
kill       1.017425
faggot     1.011733
fat        1.001284
fag        0.990578
little     0.941984
going      0.840493
time       0.805670
hope       0.759872
like       0.741675
cunt       0.715217
dtype: float64


### Looking for invalid/missing inputs

In [ ]:
df.isnull().sum()

,0
id,0
comment_text,0
toxic,0
severe_toxic,0
obscene,0
threat,0
insult,0
identity_hate,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             159571 non-null  object
 1   comment_text   159571 non-null  object
 2   toxic          159571 non-null  int64 
 3   severe_toxic   159571 non-null  int64 
 4   obscene        159571 non-null  int64 
 5   threat         159571 non-null  int64 
 6   insult         159571 non-null  int64 
 7   identity_hate  159571 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 9.7+ MB


In [ ]:
'''
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
import time
import os

# --- 1. NLTK Data Downloads (Most Robust for Colab) ---
print("Attempting NLTK data downloads to default reliable location...")

# These downloads typically place files in /root/nltk_data which is always accessible in Colab.
# We'll remove custom path logic to simplify and rely on this robust default.
try:
    nltk.download('stopwords')
    print(" 'stopwords' download/check complete.")
except Exception as e:
    print(f"Error downloading stopwords: {e}")

try:
    nltk.download('punkt')
    print(" 'punkt' download/check complete.")
except Exception as e:
    print(f"Error downloading punkt: {e}")

# --- ADD THIS LINE FOR punkt_tab ---
try:
    nltk.download('punkt_tab') # Explicitly download punkt_tab
    print(" 'punkt_tab' download/check complete.")
except Exception as e:
    print(f"Error downloading punkt_tab: {e}")
# --- END OF ADDITION ---

try:
    nltk.download('wordnet')
    print(" 'wordnet' download/check complete.")
except Exception as e:
    print(f"Error downloading wordnet: {e}")

print("NLTK data download check complete.")
print(f"NLTK data search paths: {nltk.data.path}")


# --- 2. Data Loading ---
# Load the dataset from the specified path
try:
    # Based on your directory structure screenshot, '/content/Dataset/Jigsaw/train.csv' is correct.
    df = pd.read_csv('/content/Dataset/Jigsaw/train.csv')
    print("\nDataset loaded successfully.")
    print(df.head())
except FileNotFoundError:
    print("Error: 'train.csv' not found at '/content/Dataset/Jigsaw/train.csv'.")
    print("Please ensure the file is correctly placed in your Colab environment and the path is exact (case-sensitive).")
    print("You can verify using `!ls -R /content/` in a separate cell.")
    exit()

# --- 3. Text Preprocessing ---
# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

# Define a comprehensive text preprocessing function
def preprocess_text(text):
    text = text.lower() # Convert text to lowercase
    text = re.sub(r'[^\w\s]', '', text) # Remove punctuation
    text = re.sub(r'\d+', '', text) # Remove numbers
    tokens = word_tokenize(text) # Tokenize the text
    # Lemmatize and remove stopwords
    lemmatized_tokens = [
        lemmatizer.lemmatize(word) for word in tokens if word not in stopwords.words('english')
    ]
    return ' '.join(lemmatized_tokens)

print("\nStarting text preprocessing. This may take a while...")
df['cleaned_comment_text'] = df['comment_text'].apply(preprocess_text)
print("Text preprocessing complete.")
print(df[['comment_text', 'cleaned_comment_text']].head())

# --- 4. TF-IDF Vectorization and Model Training/Evaluation for Each Label ---

target_columns = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
tfidf_vectorizer = TfidfVectorizer(max_features=5000)

results = {}

for label in target_columns:
    print(f"\n--- Processing '{label}' label ---")

    X = df['cleaned_comment_text']
    y = df[label]

    print(f"Fitting and transforming TF-IDF for '{label}'...")
    tfidf_matrix = tfidf_vectorizer.fit_transform(X)
    print(f"TF-IDF matrix shape for '{label}': {tfidf_matrix.shape}")

    # --- IMPORTANT FIX: Removed stratify=y for multi-label data ---
    # The 'stratify' parameter cannot be used with multi-label data in scikit-learn's train_test_split.
    X_train, X_test, y_train, y_test = train_test_split(
        tfidf_matrix, y, test_size=0.2, random_state=42 # Removed stratify=y
    )
    print(f"Train/Test split complete. Train samples: {X_train.shape[0]}, Test samples: {X_test.shape[0]}")

    print(f"Training XGBoost model for '{label}'...")
    xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42, n_jobs=-1)
    train_start_time = time.time()
    xgb_model.fit(X_train, y_train)
    train_end_time = time.time()
    print(f"Training complete in {train_end_time - train_start_time:.2f} seconds.")

    print(f"Evaluating model for '{label}'...")
    inference_start_time = time.time()
    y_pred = xgb_model.predict(X_test)
    y_proba = xgb_model.predict_proba(X_test)[:, 1]
    inference_end_time = time.time()

    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_proba)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    inference_time_per_sample = (inference_end_time - inference_start_time) / X_test.shape[0]

    results[label] = {
        'Accuracy': accuracy,
        'F1 Score': f1,
        'ROC-AUC': roc_auc,
        'Precision': precision,
        'Recall': recall,
        'Inference Time per Sample (s)': inference_time_per_sample
    }

    print(f"Metrics for '{label}':")
    print(f"  Accuracy: {accuracy:.4f}")
    print(f"  F1 Score: {f1:.4f}")
    print(f"  ROC-AUC: {roc_auc:.4f}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall: {recall:.4f}")
    print(f"  Inference Time per Sample: {inference_time_per_sample:.6f} seconds")

print("\n--- All models trained and evaluated ---")
print("\nSummary of results:")
for label, metrics in results.items():
    print(f"\n{label}:")
    for metric_name, value in metrics.items():
        print(f"  {metric_name}: {value:.4f}")
'''

Attempting NLTK data downloads to default reliable location...
 'stopwords' download/check complete.
 'punkt' download/check complete.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


 'punkt_tab' download/check complete.
 'wordnet' download/check complete.
NLTK data download check complete.
NLTK data search paths: ['/root/nltk_data', '/usr/nltk_data', '/usr/share/nltk_data', '/usr/lib/nltk_data', '/usr/share/nltk_data', '/usr/local/share/nltk_data', '/usr/lib/nltk_data', '/usr/local/lib/nltk_data', '/content/nltk_data']


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!



Dataset loaded successfully.
                 id                                       comment_text  toxic  \
0  0000997932d777bf  Explanation\nWhy the edits made under my usern...      0   
1  000103f0d9cfb60f  D'aww! He matches this background colour I'm s...      0   
2  000113f07ec002fd  Hey man, I'm really not trying to edit war. It...      0   
3  0001b41b1c6bb37e  "\nMore\nI can't make any real suggestions on ...      0   
4  0001d958c54c6e35  You, sir, are my hero. Any chance you remember...      0   

   severe_toxic  obscene  threat  insult  identity_hate  
0             0        0       0       0              0  
1             0        0       0       0              0  
2             0        0       0       0              0  
3             0        0       0       0              0  
4             0        0       0       0              0  

Starting text preprocessing. This may take a while...
Text preprocessing complete.
                                        comment_tex

...

Training complete in 22.68 seconds.
Evaluating model for 'threat'...
Metrics for 'threat':
  Accuracy: 0.9978
  F1 Score: 0.3107
  ROC-AUC: 0.9705
  Precision: 0.5517
  Recall: 0.2162
  Inference Time per Sample: 0.000005 seconds

--- Processing 'insult' label ---
Fitting and transforming TF-IDF for 'insult'...
TF-IDF matrix shape for 'insult': (159571, 5000)
Train/Test split complete. Train samples: 127656, Test samples: 31915
Training XGBoost model for 'insult'...


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [04:41:50] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Training complete in 44.07 seconds.
Evaluating model for 'insult'...
Metrics for 'insult':
  Accuracy: 0.9704
  F1 Score: 0.6580
  ROC-AUC: 0.9633
  Precision: 0.7911
  Recall: 0.5632
  Inference Time per Sample: 0.000006 seconds

--- Processing 'identity_hate' label ---
Fitting and transforming TF-IDF for 'identity_hate'...
TF-IDF matrix shape for 'identity_hate': (159571, 5000)
Train/Test split complete. Train samples: 127656, Test samples: 31915
Training XGBoost model for 'identity_hate'...


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [04:42:39] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Training complete in 28.36 seconds.
Evaluating model for 'identity_hate'...
Metrics for 'identity_hate':
  Accuracy: 0.9919
  F1 Score: 0.3676
  ROC-AUC: 0.9641
  Precision: 0.6579
  Recall: 0.2551
  Inference Time per Sample: 0.000006 seconds

--- All models trained and evaluated ---

Summary of results:

toxic:
  Accuracy: 0.9557
  F1 Score: 0.7228
  ROC-AUC: 0.9495
  Precision: 0.9004
  Recall: 0.6037
  Inference Time per Sample (s): 0.0000

severe_toxic:
  Accuracy: 0.9894
  F1 Score: 0.2753
  ROC-AUC: 0.9701
  Precision: 0.4444
  Recall: 0.1994
  Inference Time per Sample (s): 0.0000

obscene:
  Accuracy: 0.9798
  F1 Score: 0.7927
  ROC-AUC: 0.9783
  Precision: 0.8832
  Recall: 0.7190
  Inference Time per Sample (s): 0.0000

threat:
  Accuracy: 0.9978
  F1 Score: 0.3107
  ROC-AUC: 0.9705
  Precision: 0.5517
  Recall: 0.2162
  Inference Time per Sample (s): 0.0000

insult:
  Accuracy: 0.9704
  F1 Score: 0.6580
  ROC-AUC: 0.9633
  Precision: 0.7911
  Recall: 0.5632
  Inference Time 

In [ ]:
# updated code on 22 Jun to Include the Multiclass classifier with XGB

import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
import time
import os

# --- 1. NLTK Data Downloads ---
print("Downloading NLTK data...")
for pkg in ['stopwords', 'punkt', 'wordnet']:
    try:
        nltk.download(pkg)
        print(f" '{pkg}' download/check complete.")
    except Exception as e:
        print(f"Error downloading {pkg}: {e}")
print("NLTK data ready.\n")

# --- 2. Data Loading ---
try:
    df = pd.read_csv('/content/Dataset/Jigsaw/train.csv')
    print("Dataset loaded successfully.\n")
except FileNotFoundError:
    print("Error: train.csv not found.")
    exit()

# --- 3. Text Preprocessing ---
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    tokens = word_tokenize(text)
    return ' '.join(
        lemmatizer.lemmatize(tok)
        for tok in tokens
        if tok not in stopwords.words('english')
    )

print("Preprocessing text (this may take a moment)...")
df['cleaned_comment_text'] = df['comment_text'].apply(preprocess_text)
print("Text preprocessing complete.\n")

# --- 4. TF-IDF Vectorization once for all labels ---
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X = tfidf_vectorizer.fit_transform(df['cleaned_comment_text'])
print(f"TF-IDF matrix shape: {X.shape}\n")

# --- 5. Prepare multi-label targets & split ---
target_columns = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
y = df[target_columns]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
print(f"Train/test split: {X_train.shape[0]} train samples, {X_test.shape[0]} test samples.\n")

# --- 6. MultiOutput XGBoost Training ---
base_xgb  = XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42,
    n_jobs=-1
)
multi_xgb = MultiOutputClassifier(base_xgb, n_jobs=-1)

print("Training MultiOutput XGBoost on all labels...")
t0 = time.time()
multi_xgb.fit(X_train, y_train)
print(f"Training completed in {time.time() - t0:.2f} seconds.\n")

# --- 7. Predictions & Probabilities ---
print("Generating predictions and probabilities...")
y_pred = multi_xgb.predict(X_test)
# collect probability-of-positive for each label
proba_list = [
    est.predict_proba(X_test)[:, 1]
    for est in multi_xgb.estimators_
]
y_proba = np.vstack(proba_list).T
print("Done.\n")

# --- 8. Evaluation Metrics per Label ---
results = {}
for idx, label in enumerate(target_columns):
    acc      = accuracy_score(y_test[label], y_pred[:, idx])
    f1       = f1_score(y_test[label],    y_pred[:, idx], zero_division=0)
    roc_auc  = roc_auc_score(y_test[label], y_proba[:, idx])
    prec     = precision_score(y_test[label], y_pred[:, idx], zero_division=0)
    rec      = recall_score(y_test[label],    y_pred[:, idx], zero_division=0)
    results[label] = {
        'Accuracy': acc,
        'F1 Score': f1,
        'ROC-AUC': roc_auc,
        'Precision': prec,
        'Recall': rec
    }

# --- 9. Print Summary ---
print("--- Multi-Label XGBoost Results ---")
for label, metrics in results.items():
    print(f"\nLabel: {label}")
    for name, val in metrics.items():
        print(f"  {name:25s}: {val:.4f}")


 'stopwords' download/check complete.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


 'punkt' download/check complete.
 'wordnet' download/check complete.
NLTK data ready.

Dataset loaded successfully.

Preprocessing text (this may take a moment)...
Text preprocessing complete.

TF-IDF matrix shape: (159571, 5000)

Train/test split: 127656 train samples, 31915 test samples.

Training MultiOutput XGBoost on all labels...
Training completed in 185.84 seconds.

Generating predictions and probabilities...
Done.

--- Multi-Label XGBoost Results ---

Label: toxic
  Accuracy                 : 0.9557
  F1 Score                 : 0.7228
  ROC-AUC                  : 0.9495
  Precision                : 0.9004
  Recall                   : 0.6037

Label: severe_toxic
  Accuracy                 : 0.9894
  F1 Score                 : 0.2753
  ROC-AUC                  : 0.9701
  Precision                : 0.4444
  Recall                   : 0.1994

Label: obscene
  Accuracy                 : 0.9798
  F1 Score                 : 0.7927
  ROC-AUC                  : 0.9783
  Precision     

In [ ]:
new_comments = [
  "That user is a complete idiot!",
  "I enjoyed your post, very helpful."
]

# 1) Preprocess exactly as before
cleaned = [preprocess_text(c) for c in new_comments]

# 2) TF-IDF transform (note: **transform**, not fit_transform)
X_new = tfidf_vectorizer.transform(cleaned)

# 3) Get binary predictions and probabilities
pred_labels = multi_xgb.predict(X_new)            # shape (n_new, 6)
pred_probs  = np.vstack([
  est.predict_proba(X_new)[:,1]
  for est in multi_xgb.estimators_
]).T                                              # shape (n_new, 6)

# 4) Map back to category names
for comment, labels_row, probs_row in zip(new_comments, pred_labels, pred_probs):
    print(f"\nComment: {comment}")
    for cat, lbl, prob in zip(target_cols, labels_row, probs_row):
        print(f"  {cat:15s}: pred={lbl}  prob={prob:.3f}")



Comment: That user is a complete idiot!
  toxic          : pred=1  prob=0.835
  severe_toxic   : pred=0  prob=0.006
  obscene        : pred=0  prob=0.302
  threat         : pred=0  prob=0.001
  insult         : pred=1  prob=0.741
  identity_hate  : pred=0  prob=0.006

Comment: I enjoyed your post, very helpful.
  toxic          : pred=0  prob=0.051
  severe_toxic   : pred=0  prob=0.003
  obscene        : pred=0  prob=0.015
  threat         : pred=0  prob=0.001
  insult         : pred=0  prob=0.020
  identity_hate  : pred=0  prob=0.003


In [ ]:
# -----------------------------------------------
# 1. Clean & Install Compatible Libraries
# -----------------------------------------------
!pip uninstall -y transformers accelerate sentence-transformers
!pip install -q transformers[torch]==4.44.0 datasets==2.10.1 torch scikit-learn

import os
os.environ["WANDB_DISABLED"] = "true"    # disable Weights & Biases logging

import pandas as pd
import numpy as np
import torch
from transformers import (
    BertTokenizerFast,
    BertForSequenceClassification,
    Trainer,
    TrainingArguments
)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from datasets import Dataset
import time

# -----------------------------------------------
# 2. Load & Prepare Data
# -----------------------------------------------
df = pd.read_csv('/content/Dataset/Jigsaw/train.csv')
target_cols = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']
df[target_cols] = df[target_cols].astype(float)
df['label'] = (df[target_cols].sum(axis=1) > 0).astype(int)

# -----------------------------------------------
# 3. Tokenization & Dataset Creation
# -----------------------------------------------
MODEL_NAME = 'bert-base-uncased'
MAX_LEN    = 128
tokenizer  = BertTokenizerFast.from_pretrained(MODEL_NAME)

def tokenize_fn(examples):
    return tokenizer(
        examples['comment_text'],
        truncation=True,
        padding='max_length',
        max_length=MAX_LEN
    )

dataset = Dataset.from_pandas(df[['comment_text','label']])
dataset = dataset.map(tokenize_fn, batched=True)
dataset = dataset.rename_column('label','labels')
dataset.set_format(type='torch', columns=['input_ids','attention_mask','labels'])

splits       = dataset.train_test_split(test_size=0.2, seed=42)
train_ds     = splits['train']
eval_ds      = splits['test']

# -----------------------------------------------
# 4. Model & Trainer Setup
# -----------------------------------------------
model = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = torch.argmax(torch.tensor(logits), dim=1)
    prec, rec, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc, 'f1': f1, 'precision': prec, 'recall': rec}

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    eval_strategy='epoch',
    save_strategy='epoch',
    logging_dir='./logs',
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    report_to=[]                    # disable all loggers (no wandb)
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# -----------------------------------------------
# 5. Train
# -----------------------------------------------
print("⏳ Training the BERT model...")
start = time.time()
trainer.train()
print("✅ Training completed in", time.time() - start, "seconds")

# -----------------------------------------------
# 6. Evaluate
# -----------------------------------------------
print("🔍 Evaluating on validation set...")
metrics = trainer.evaluate()
print(metrics)

# -----------------------------------------------
# 7. Inference Time Check
# -----------------------------------------------
sample_texts = ["You are a stupid idiot", "Thank you, this was helpful!"]
inputs = tokenizer(sample_texts, padding=True, truncation=True, return_tensors='pt')
inputs = {k: v.to(model.device) for k, v in inputs.items()}

start_inf = time.time()
with torch.no_grad():
    _ = model(**inputs)
print(f"\nInference time for {len(sample_texts)} samples: {time.time() - start_inf:.6f} sec")

print("--- BERT Model Training and Inference Complete ---")


Found existing installation: transformers 4.44.0
Uninstalling transformers-4.44.0:
  Successfully uninstalled transformers-4.44.0
Found existing installation: accelerate 1.7.0
Uninstalling accelerate-1.7.0:
  Successfully uninstalled accelerate-1.7.0


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/159571 [00:00<?, ? examples/s]

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

⏳ Training the BERT model...


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.147900,0.323881,0.900298,0.012415,1.000000,0.006246
2,0.335000,0.328785,0.899671,0.000000,0.000000,0.000000


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.147900,0.323881,0.900298,0.012415,1.000000,0.006246
2,0.335000,0.328785,0.899671,0.000000,0.000000,0.000000
3,0.137200,0.148128,0.962369,0.808971,0.824311,0.794191


✅ Training completed in 9104.351750850677 seconds
🔍 Evaluating on validation set...


{'eval_loss': 0.14812788367271423, 'eval_accuracy': 0.9623687921040264, 'eval_f1': 0.8089708923174805, 'eval_precision': 0.8243111831442463, 'eval_recall': 0.7941911305434104, 'eval_runtime': 225.003, 'eval_samples_per_second': 141.843, 'eval_steps_per_second': 8.867, 'epoch': 3.0}

Inference time for 2 samples: 0.023408 sec
--- BERT Model Training and Inference Complete ---


In [ ]:
# -----------------------------------------------
# 1. Clean & Install Compatible Libraries
# -----------------------------------------------
!pip uninstall -y transformers accelerate sentence-transformers
!pip install -q transformers[torch]==4.44.0 torch scikit-learn

import os
os.environ["WANDB_DISABLED"] = "true"    # turn off W&B

import time
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset as TorchDataset
from transformers import (
    BertTokenizerFast,
    BertForSequenceClassification,
    Trainer,
    TrainingArguments
)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split

# -----------------------------------------------
# 2. Load & Prepare DataFrame
# -----------------------------------------------
df = pd.read_csv('/content/Dataset/Jigsaw/train.csv')
target_cols = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']
df[target_cols] = df[target_cols].astype(float)
df['label'] = (df[target_cols].sum(axis=1) > 0).astype(int)

# -----------------------------------------------
# 3. Tokenize All Text at Once (returns torch.Tensor)
# -----------------------------------------------
MODEL_NAME = 'bert-base-uncased'
MAX_LEN    = 128
tokenizer  = BertTokenizerFast.from_pretrained(MODEL_NAME)

encodings = tokenizer(
    df['comment_text'].tolist(),
    truncation=True,
    padding='max_length',
    max_length=MAX_LEN,
    return_tensors='pt'
)
labels = torch.tensor(df['label'].values)

# -----------------------------------------------
# 4. Train/Test Split of Indices
# -----------------------------------------------
train_idx, val_idx = train_test_split(
    np.arange(len(labels)),
    test_size=0.2,
    stratify=labels,
    random_state=42
)

# -----------------------------------------------
# 5. Build pure PyTorch Datasets
# -----------------------------------------------
class ToxDataset(TorchDataset):
    def __init__(self, encodings, labels, indices):
        self.encodings = encodings
        self.labels    = labels
        self.idx       = indices
    def __len__(self):
        return len(self.idx)
    def __getitem__(self, i):
        j = self.idx[i]
        return {
            'input_ids':      self.encodings.input_ids[j],
            'attention_mask': self.encodings.attention_mask[j],
            'labels':         self.labels[j]
        }

train_dataset = ToxDataset(encodings, labels, train_idx)
eval_dataset  = ToxDataset(encodings, labels, val_idx)

# -----------------------------------------------
# 6. Compute Class Weights for Imbalance
# -----------------------------------------------
class_weights = compute_class_weight(
    'balanced',
    classes=np.array([0,1]),
    y=df['label']
)
class_weights = torch.tensor(class_weights, dtype=torch.float).to('cuda' if torch.cuda.is_available() else 'cpu')

# -----------------------------------------------
# 7. Model + Weighted Trainer Subclass
# -----------------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model  = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2).to(device)

class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels").to(device)
        inputs = {k: v.to(device) for k,v in inputs.items()}
        outputs = model(**inputs)
        logits  = outputs.logits
        loss_fct = nn.CrossEntropyLoss(weight=class_weights)
        loss     = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = torch.softmax(torch.tensor(logits), dim=1)[:,1].numpy()
    preds = (probs > 0.5).astype(int)
    return {
        'accuracy': accuracy_score(labels, preds),
        'roc_auc':  roc_auc_score(labels, probs),
        **{
            m: v for m, v in zip(
                ['precision','recall','f1','_'],
                precision_recall_fscore_support(
                    labels, preds, average='binary', zero_division=0
                )
            )
        }
    }

# -----------------------------------------------
# 8. Setup Trainer
# -----------------------------------------------
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    eval_strategy='epoch',
    save_strategy='epoch',
    logging_dir='./logs',
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    report_to=[]    # no wandb
)

trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# -----------------------------------------------
# 9. Train & Evaluate
# -----------------------------------------------
print("⏳ Training...")
t0 = time.time()
trainer.train()
print("✅ Training done in", time.time() - t0, "sec")

print("🔍 Evaluating...")
print(trainer.evaluate())

# -----------------------------------------------
# 10. Quick Inference Check
# -----------------------------------------------
sample_texts = [
    "You are a stupid idiot",
    "Thank you, this was helpful!"
]
inputs = tokenizer(sample_texts, padding=True, truncation=True, return_tensors='pt')
inputs = {k:v.to(device) for k,v in inputs.items()}

t1 = time.time()
with torch.no_grad():
    _ = model(**inputs)
print(f"Inference for {len(sample_texts)} samples: {time.time()-t1:.6f} sec")
print("--- All done ---")


Found existing installation: transformers 4.44.0
Uninstalling transformers-4.44.0:
  Successfully uninstalled transformers-4.44.0
Found existing installation: accelerate 1.8.1
Uninstalling accelerate-1.8.1:
  Successfully uninstalled accelerate-1.8.1


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please u

⏳ Training...


Epoch,Training Loss,Validation Loss,Accuracy,Roc Auc,Precision,Recall,F1,
1,0.396600,0.411596,0.964029,0.975727,0.855544,0.777504,0.814659,None
2,0.362100,0.495530,0.962306,0.798747,0.861800,0.749461,0.801714,None


Epoch,Training Loss,Validation Loss,Accuracy,Roc Auc,Precision,Recall,F1,
1,0.396600,0.411596,0.964029,0.975727,0.855544,0.777504,0.814659,None
2,0.362100,0.495530,0.962306,0.798747,0.861800,0.749461,0.801714,None
3,0.237700,0.362439,0.962024,0.977187,0.818952,0.804314,0.811567,None


✅ Training done in 8908.92094707489 sec
🔍 Evaluating...


{'eval_loss': 0.4115961492061615, 'eval_accuracy': 0.9640294532351559, 'eval_roc_auc': 0.9757274022496041, 'eval_precision': 0.8555442522889115, 'eval_recall': 0.7775038520801233, 'eval_f1': 0.8146593477558928, 'eval__': None, 'eval_runtime': 217.0962, 'eval_samples_per_second': 147.009, 'eval_steps_per_second': 9.189, 'epoch': 3.0}
Inference for 2 samples: 0.050874 sec
--- All done ---
